In [1]:
import urllib
import os
import operator
import pathlib
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
%matplotlib inline

In [2]:
df_age = pd.read_excel('../data/raw/boston/neighborhood-demographics-2013-2017-acs.xlsx', sheet_name='Age', skiprows=2)

In [3]:
df_age.head(3)

,Unnamed: 0,Total Population,Median Age,0-9 years,%,10-19 years,%.1,20-34 years,%.2,35-54 years,%.3,55-64 years,%.4,65 years and over,%.5
0,United States,321004407.0,38.0,40298637.0,0.125539,41932161.0,0.130628,66546138.0,0.207306,83747562.0,0.260892,40747520.0,0.126938,47732389.0,0.148697
1,Massachusetts,6789319.0,39.0,734048.0,0.108118,858569.0,0.126459,1437244.0,0.211692,1807990.0,0.266299,901717.0,0.132814,1049751.0,0.154618
2,Boston,669158.0,32.0,63428.0,0.094788,76451.0,0.114250,231255.0,0.345591,157768.0,0.235771,66662.0,0.099621,73594.0,0.109980


In [4]:
df_age.rename(columns={
    'Unnamed: 0':'Location',
    'Total Population':'Total-Population',
    'Median Age':'Median-Age',
    '0-9 years':'0:9-Years',
    '%':'0:9-Percentage',
    '10-19 years':'10:19-Years',
    '%.1':'10:19-Percentage',
    '20-34 years': '20:34-Years',
    '%.2': '20:34-Percentage',
    '35-54 years': '35:54-Years',
    '%.3': '35:54-Percentage',
    '55-64 years': '55:64-Years',
    '%.4': '55:64-Percentage',
    '65 years and over ': '65+Years',
    '%.5': '65+Percentage'
    }, 
    inplace=True)

In [5]:
df_age.head(10)

,Location,Total-Population,Median-Age,0:9-Years,0:9-Percentage,10:19-Years,10:19-Percentage,20:34-Years,20:34-Percentage,35:54-Years,35:54-Percentage,55-64 years,55:64-Percentage,65+Years,65+Percentage
0,United States,321004407.0,38.0,40298637.0,0.125539,41932161.0,0.130628,66546138.0,0.207306,83747562.0,0.260892,40747520.0,0.126938,47732389.0,0.148697
1,Massachusetts,6789319.0,39.0,734048.0,0.108118,858569.0,0.126459,1437244.0,0.211692,1807990.0,0.266299,901717.0,0.132814,1049751.0,0.154618
2,Boston,669158.0,32.0,63428.0,0.094788,76451.0,0.114250,231255.0,0.345591,157768.0,0.235771,66662.0,0.099621,73594.0,0.109980
3,Allston,19363.0,26.0,550.0,0.028405,2691.0,0.138976,13084.0,0.675722,1711.0,0.088364,675.0,0.034860,652.0,0.033672
4,Back Bay,18176.0,33.0,798.0,0.043904,1632.0,0.089789,7498.0,0.412522,3498.0,0.192452,2118.0,0.116527,2632.0,0.144806
5,Beacon Hill,9751.0,32.0,728.0,0.074659,322.0,0.033022,4758.0,0.487950,1826.0,0.187263,921.0,0.094452,1196.0,0.122654
6,Brighton,51785.0,29.0,3271.0,0.063165,3244.0,0.062644,27399.0,0.529091,8337.0,0.160993,3340.0,0.064497,6194.0,0.119610
7,Charlestown,18901.0,35.0,2424.0,0.128247,1196.0,0.063277,5738.0,0.303582,5315.0,0.281202,2268.0,0.119994,1960.0,0.103698
8,Dorchester,125947.0,33.0,15270.0,0.121241,16381.0,0.130063,35301.0,0.280285,32714.0,0.259744,13047.0,0.103591,13234.0,0.105076
9,Downtown,17581.0,33.0,929.0,0.052841,2666.0,0.151641,5888.0,0.334907,3544.0,0.201581,2023.0,0.115067,2531.0,0.143962


In [6]:
#Removing aggregated fields {United States, Massachusetts, Boston}
df_age_new = df_age[3:]
df_age_new.head()

,Location,Total-Population,Median-Age,0:9-Years,0:9-Percentage,10:19-Years,10:19-Percentage,20:34-Years,20:34-Percentage,35:54-Years,35:54-Percentage,55-64 years,55:64-Percentage,65+Years,65+Percentage
3,Allston,19363.0,26.0,550.0,0.028405,2691.0,0.138976,13084.0,0.675722,1711.0,0.088364,675.0,0.034860,652.0,0.033672
4,Back Bay,18176.0,33.0,798.0,0.043904,1632.0,0.089789,7498.0,0.412522,3498.0,0.192452,2118.0,0.116527,2632.0,0.144806
5,Beacon Hill,9751.0,32.0,728.0,0.074659,322.0,0.033022,4758.0,0.487950,1826.0,0.187263,921.0,0.094452,1196.0,0.122654
6,Brighton,51785.0,29.0,3271.0,0.063165,3244.0,0.062644,27399.0,0.529091,8337.0,0.160993,3340.0,0.064497,6194.0,0.119610
7,Charlestown,18901.0,35.0,2424.0,0.128247,1196.0,0.063277,5738.0,0.303582,5315.0,0.281202,2268.0,0.119994,1960.0,0.103698


In [7]:
#Bring in housing tenure
df_housing_tenure = pd.read_excel('../data/raw/boston/neighborhood-demographics-2013-2017-acs.xlsx', sheet_name='Housing Tenure', skiprows=2)


In [8]:
df_housing_tenure.head(3)
df_housing_tenure.columns

Index(['Unnamed: 0', 'Total Housing Units: ', 'Total Occupied',
       '% of Total Housing Units', 'Owner Occupied', '% of Occupied Units',
       'Renter Occupied', '% of Occupied Units.1'],
      dtype='object')

In [9]:
df_housing_tenure.rename(columns={
    'Unnamed: 0':'Location',
    'Total Housing Units':'Total-Housing-Units',
    'Total Occupied':'Total-Occupied',
    '% of Total Housing Units':'Housing-Unit-Percentage',
    'Owner Occupied':'Owner-Occupied',
    '% of Occupied Units':'Owner-Occupied-Units-Percentage',
    'Renter Occupied':'Renter-Occupied',
    '% of Occupied Units.1': 'Renter-Occupied-Units-Percentage'
    }, 
    inplace=True)

In [10]:
df_housing_tenure.head(3)

,Location,Total Housing Units:,Total-Occupied,Housing-Unit-Percentage,Owner-Occupied,Owner-Occupied-Units-Percentage,Renter-Occupied,Renter-Occupied-Units-Percentage
0,United States,135393564.0,118825921.0,0.877633,75833135.0,0.638187,42992786.0,0.361813
1,Massachusetts,2864989.0,2585715.0,0.902522,1612329.0,0.623552,973386.0,0.376448
2,Boston,285660.0,263229.0,0.921477,93046.0,0.353479,170183.0,0.647000


In [11]:
#Removing aggregated fields {United States, Massachusetts, Boston}
df_housing_tenure_new = df_housing_tenure[3:]
df_housing_tenure_new.head(3)

,Location,Total Housing Units:,Total-Occupied,Housing-Unit-Percentage,Owner-Occupied,Owner-Occupied-Units-Percentage,Renter-Occupied,Renter-Occupied-Units-Percentage
3,Allston,7110.0,6457.0,0.908,631.0,0.098,5826.0,0.902
4,Back Bay,11773.0,9824.0,0.834,3196.0,0.325,6628.0,0.675
5,Beacon Hill,6014.0,5458.0,0.908,1849.0,0.339,3609.0,0.661


In [12]:
#Bringing in house hold income
df_hh_income = pd.read_excel('../data/raw/boston/neighborhood-demographics-2013-2017-acs.xlsx', sheet_name='HH Income', skiprows=2)

In [13]:
df_hh_income.head(3)
df_hh_income.columns

Index(['Unnamed: 0', 'Median Income', 'Total Households', '$14,999 and under',
       '%', '$15,000 to $24,999', '%.1', '$25,000 to $34,999', '%.2',
       '$35,000 to $49,999', '%.3', '$50,000 to $74,999', '%.4',
       '$75,000 to $99,999', '%.5', '$100,000 to $149,999', '%.6',
       '$150,000 +', '%.7'],
      dtype='object')

In [14]:
#Fixing column headings
df_hh_income.rename(columns={
    'Unnamed: 0':'Location',
    'Median Income':'Median-Income',
    'Total Households':'Total-Households',
    '$14,999 and under':'14999-And-Under',
    '%':'14999-And-Under-Percentage',
    '$15,000 to $24,999':'15-24999K',
    '%.1':'15-24999K-Percentage',
    '$25,000 to $34,999': '25-34999K',
    '%.2': '25-34999K-Percentage',
    '$35,000 to $49,999': '35-49999K',
    '%.3': '35-49999K-Percentage',
    '$50,000 to $74,999': '50-74999K',
    '%.4': '50-74999K-Percentage',
    '$75,000 to $99,999': '75-99999K',
    '%.5': '75-99999K-Percentage',
    '$100,000 to $149,999': '100-149999K',
    '%.6': '100-149999K-Percentage',
    '$150,000 +': '150K+',
    '%.7': '150K+Percentage'
    }, 
    inplace=True)

In [15]:
#Removing aggregated fields {United States, Massachusetts, Boston}
df_hh_income_new = df_hh_income[3:]
df_hh_income_new.head(3)

,Location,Median-Income,Total-Households,14999-And-Under,14999-And-Under-Percentage,15-24999K,15-24999K-Percentage,25-34999K,25-34999K-Percentage,35-49999K,35-49999K-Percentage,50-74999K,50-74999K-Percentage,75-99999K,75-99999K-Percentage,100-149999K,100-149999K-Percentage,150K+,150K+Percentage
3,Allston,46982.76,6457.0,1453.0,0.225027,584.0,0.090444,603.0,0.093387,781.0,0.120954,1042.0,0.161375,544.0,0.084250,912.0,0.141242,538.0,0.083320
4,Back Bay,102070.55,9824.0,1402.0,0.142712,381.0,0.038783,350.0,0.035627,582.0,0.059243,1237.0,0.125916,879.0,0.089475,1586.0,0.161441,3407.0,0.346804
5,Beacon Hill,98069.24,5458.0,582.0,0.106632,205.0,0.037560,159.0,0.029132,360.0,0.065958,730.0,0.133749,751.0,0.137596,939.0,0.172041,1732.0,0.317332


In [16]:
#Bring in Poverty Rate
df_poverty_rates = pd.read_excel('../data/raw/boston/neighborhood-demographics-2013-2017-acs.xlsx', sheet_name='Poverty Rates', skiprows=2)

In [17]:
df_poverty_rates.head(3)
df_poverty_rates.columns

Index(['Unnamed: 0', 'Total population for whom poverty status is determined',
       'Total in poverty', 'Poverty rate', 'Percent of Boston's impoverished'],
      dtype='object')

In [18]:
#Fixing column headings
df_poverty_rates.rename(columns={
    'Unnamed: 0':'Location',
    'Total population for whom poverty status is determined':'Total-Population',
    'Total in poverty':'Total-In-Poverty',
    'Poverty rate':'Poverty-Rate',
    'Percent of Boston\'s impoverished':'Percent-Boston-Impoverished',
    }, 
    inplace=True)

In [19]:
#Removing aggregated fields {United States, Massachusetts, Boston}
df_poverty_rates_new = df_poverty_rates[3:]
df_poverty_rates_new.head(3)

,Location,Total-Population,Total-In-Poverty,Poverty-Rate,Percent-Boston-Impoverished
3,Allston,13892.0,4326.0,0.311402,0.033634
4,Back Bay,16661.0,1958.0,0.117520,0.015223
5,Beacon Hill,9751.0,907.0,0.093016,0.007052


In [20]:
#Bring in Educational Attainment
df_educational_attainment = pd.read_excel('../data/raw/boston/neighborhood-demographics-2013-2017-acs.xlsx', sheet_name='Educational Attainment', skiprows=2)

In [21]:
df_educational_attainment.head(3)
df_educational_attainment.columns

Index(['Unnamed: 0', 'Total population 25 years and over',
       'Less than High School ', '%', 'High School Graduate', '%.1',
       'Some College ', '%.2', 'Bachelor's Degree or more ', '%.3'],
      dtype='object')

In [22]:
#Fixing column headings
df_educational_attainment.rename(columns={
    'Unnamed: 0':'Location',
    'Total population 25 years and over':'Total-Population-25+',
    'Less than High School':'Less-Than-High-School',
    '%':'Less-Than-High-School-Percentage',
    'High School Graduate':'High-School-Graduate',
    '%.1':'High-School-Graduate-Percentage',
    'Some College':'Some-College',
    '%.2': 'Some-College-Percentage',
    'Bachelor\'s Degree or more ': 'Bachelor-Degree-Or-More',
    '%.3': 'Bachelor-Degree-Or-More-Percentage'
    }, 
    inplace=True)

In [23]:
df_educational_attainment.head(3)

,Location,Total-Population-25+,Less than High School,Less-Than-High-School-Percentage,High-School-Graduate,High-School-Graduate-Percentage,Some College,Some-College-Percentage,Bachelor-Degree-Or-More,Bachelor-Degree-Or-More-Percentage
0,United States,216271644.0,27437114.0,0.126864,59093612.0,0.273238,62853315.0,0.290622,66887603.0,0.309276
1,Massachusetts,4706536.0,458080.0,0.097328,1162683.0,0.247036,1104912.0,0.234761,1980861.0,0.420875
2,Boston,457113.0,63725.0,0.139408,95687.0,0.209329,81074.0,0.177361,216627.0,0.473903


In [24]:
#Removing aggregated fields {United States, Massachusetts, Boston}
df_educational_attainment_new = df_educational_attainment[3:]
df_educational_attainment_new.head(3)

,Location,Total-Population-25+,Less than High School,Less-Than-High-School-Percentage,High-School-Graduate,High-School-Graduate-Percentage,Some College,Some-College-Percentage,Bachelor-Degree-Or-More,Bachelor-Degree-Or-More-Percentage
3,Allston,10561.0,715.0,0.067702,986.0,0.093362,1265.0,0.119780,7595.0,0.719155
4,Back Bay,14166.0,324.0,0.022872,699.0,0.049343,1095.0,0.077298,12048.0,0.850487
5,Beacon Hill,7948.0,182.0,0.022899,176.0,0.022144,455.0,0.057247,7135.0,0.897710


In [25]:
#Bring in School Enrollment
df_school_enrollment = pd.read_excel('../data/raw/boston/neighborhood-demographics-2013-2017-acs.xlsx', sheet_name='School Enrollment', skiprows=2)

In [26]:
df_school_enrollment.head(3)
df_school_enrollment.columns

Index(['Unnamed: 0', 'Population Age 3+', 'Enrolled in school:', '% of Age 3+',
       'Enrolled in nursery school, preschool', '% of Enrolled',
       'Enrolled in kindergarten', '% of Enrolled.1',
       'Enrolled in grade 1 to grade 4', '% of Enrolled.2',
       'Enrolled in grade 5 to grade 8', '% of Enrolled.3',
       'Enrolled in grade 9 to grade 12', '% of Enrolled.4',
       'Enrolled in college, undergraduate years', '% of Enrolled.5',
       'Graduate or professional school', '% of Enrolled.6',
       'Not enrolled in school', '% of Total'],
      dtype='object')

In [27]:
#Fixing column headings
df_school_enrollment.rename(columns={
    'Unnamed: 0':'Location',
    'Population Age 3+':'Population-Age-3+',
    'Enrolled in school:':'Enrolled-In-School:',
    '% of Age 3+': '3+-Percentage',
    'Enrolled in nursery school, preschool': 'Enrolled-Nursery-School-Preschool',
    '% of Enrolled': 'Enrolled-Nursery-School-Preschool-Percentage',
    'Enrolled in kindergarten': 'Enrolled-In-Kindergarten',
    '% of Enrolled.1': 'Enrolled-In-Kindergarten-Percentage',
    'Enrolled in grade 1 to grade 4': 'Enrolled-Grade-1:4',
    '% of Enrolled.2': 'Enrolled-Grade-1:4-Percentage',
    'Enrolled in grade 5 to grade 8': 'Enrolled-Grade-5:8',
    '% of Enrolled.3': 'Enrolled-Grade-5:8-Percentage',
    'Enrolled in grade 9 to grade 12': 'Enrolled-Grade-9:12',
    '% of Enrolled.4': 'Enrolled-Grade-9:12-Percentage',
    'Enrolled in college, undergraduate years': 'Enrolled-College-Undergraduate',
    '% of Enrolled.5': 'Enrolled-College-Undergraduate-Percentage',
    'Graduate or professional school': 'Graduate-Or-Professional-School',
    '% of Enrolled.6': 'Graduate-Or-Professional-School-Percentage',
    'Not enrolled in school': 'Not-Enrolled-In-School',
    '% of Total': 'Not-Enrolled-In-School-Percentage'
    }, 
    inplace=True)

In [28]:
df_school_enrollment.head(3)

,Location,Population-Age-3+,Enrolled-In-School:,3+-Percentage,Enrolled-Nursery-School-Preschool,Enrolled-Nursery-School-Preschool-Percentage,Enrolled-In-Kindergarten,Enrolled-In-Kindergarten-Percentage,Enrolled-Grade-1:4,Enrolled-Grade-1:4-Percentage,Enrolled-Grade-5:8,Enrolled-Grade-5:8-Percentage,Enrolled-Grade-9:12,Enrolled-Grade-9:12-Percentage,Enrolled-College-Undergraduate,Enrolled-College-Undergraduate-Percentage,Graduate-Or-Professional-School,Graduate-Or-Professional-School-Percentage,Not-Enrolled-In-School,Not-Enrolled-In-School-Percentage
0,United States,309341395.0,81751797.0,0.264277,4934251.0,0.060356,4136743.0,0.050601,16335701.0,0.199821,16495557.0,0.201776,17001421.0,0.207964,18697600.0,0.228712,4150524.0,0.050770,227589598.0,0.735723
1,Massachusets,6576630.0,1737576.0,0.264205,110204.0,0.063424,77478.0,0.044590,298673.0,0.171891,316676.0,0.182252,343788.0,0.197855,450807.0,0.259446,139950.0,0.080543,4839054.0,0.735795
2,Boston,647974.0,191254.0,0.295157,7606.0,0.039769,8621.0,0.045076,22689.0,0.118633,22435.0,0.117305,24627.0,0.128766,75360.0,0.394031,29916.0,0.156420,456720.0,0.704843


In [29]:
#Removing aggregated fields {United States, Massachusetts, Boston}
df_school_enrollment_new = df_school_enrollment[3:]
df_school_enrollment_new.head(3)

,Location,Population-Age-3+,Enrolled-In-School:,3+-Percentage,Enrolled-Nursery-School-Preschool,Enrolled-Nursery-School-Preschool-Percentage,Enrolled-In-Kindergarten,Enrolled-In-Kindergarten-Percentage,Enrolled-Grade-1:4,Enrolled-Grade-1:4-Percentage,Enrolled-Grade-5:8,Enrolled-Grade-5:8-Percentage,Enrolled-Grade-9:12,Enrolled-Grade-9:12-Percentage,Enrolled-College-Undergraduate,Enrolled-College-Undergraduate-Percentage,Graduate-Or-Professional-School,Graduate-Or-Professional-School-Percentage,Not-Enrolled-In-School,Not-Enrolled-In-School-Percentage
3,Allston,19174.0,11318.0,0.590279,34.0,0.003004,38.0,0.003357,271.0,0.023944,162.0,0.014313,77.0,0.006803,6630.0,0.585793,4106.0,0.362785,7856.0,0.409721
4,Back Bay,17820.0,4454.0,0.249944,150.0,0.033678,45.0,0.010103,247.0,0.055456,227.0,0.050965,171.0,0.038392,2321.0,0.521105,1293.0,0.290301,13366.0,0.750056
5,Beacon Hill,9348.0,1504.0,0.160890,157.0,0.104388,41.0,0.027261,206.0,0.136968,67.0,0.044548,71.0,0.047207,503.0,0.334441,459.0,0.305186,7844.0,0.839110


In [53]:
# combine dfs into a list to make final cleansing steps easier to complete
df_name_list = [
    'df_age_new',
    'df_housing_tenure_new',
    'df_hh_income_new',
    'df_poverty_rates_new',
    'df_educational_attainment_new',
    'df_school_enrollment_new'
] 

In [55]:
# remove excess rows and strip trailing spaces from location names 
for df_name in df_name_list:
    globals()[df_name] = globals()[df_name].copy().iloc[:-3, :]
    globals()[df_name]['Location'] = globals()[df_name]['Location'].str.strip()

In [57]:
#Create master dataframe / .csv for future use:
#df_master_demo = pd.merge(df_age_new, df_housing_tenure_new, df_hh_income_new, df_poverty_rates_new
#                          df_educational_attainment_new, df_school_enrollment_new, on='Location')

df_master_demo1 = pd.merge(df_age_new, df_housing_tenure_new, on='Location')
df_master_demo2 = pd.merge(df_master_demo1, df_hh_income_new, on='Location')
df_master_demo3 = pd.merge(df_master_demo2, df_poverty_rates_new, on='Location')
df_master_demo4 = pd.merge(df_master_demo3, df_educational_attainment_new, on='Location')
df_master_demo_final = pd.merge(df_master_demo4, df_school_enrollment_new, on='Location')

print(df_master_demo_final.shape)
df_master_demo_final.head(3)

(23, 72)


,Location,Total-Population_x,Median-Age,0:9-Years,0:9-Percentage,10:19-Years,10:19-Percentage,20:34-Years,20:34-Percentage,35:54-Years,...,Enrolled-Grade-5:8,Enrolled-Grade-5:8-Percentage,Enrolled-Grade-9:12,Enrolled-Grade-9:12-Percentage,Enrolled-College-Undergraduate,Enrolled-College-Undergraduate-Percentage,Graduate-Or-Professional-School,Graduate-Or-Professional-School-Percentage,Not-Enrolled-In-School,Not-Enrolled-In-School-Percentage
0,Allston,19363.0,26.0,550.0,0.028405,2691.0,0.138976,13084.0,0.675722,1711.0,...,162.0,0.014313,77.0,0.006803,6630.0,0.585793,4106.0,0.362785,7856.0,0.409721
1,Back Bay,18176.0,33.0,798.0,0.043904,1632.0,0.089789,7498.0,0.412522,3498.0,...,227.0,0.050965,171.0,0.038392,2321.0,0.521105,1293.0,0.290301,13366.0,0.750056
2,Beacon Hill,9751.0,32.0,728.0,0.074659,322.0,0.033022,4758.0,0.487950,1826.0,...,67.0,0.044548,71.0,0.047207,503.0,0.334441,459.0,0.305186,7844.0,0.839110


In [59]:
#Save df_master_demo_final to file
df_master_demo_final.to_csv('../data/interim/demographics_master_neighborhood.csv', sep=',')
